### Prepping Data Challenge: NBA Draft Lottery calculator (week 27)

### Challenge
We want you to calculate the results of the NBA draft lottery for 2021. The flow should be able to re-run to recalculate the results as if the lottery is done again with each run. 

The lottery gives teams the chance to land the best picks in the NBA draft. The NBA draft is where teams take it in turns to allocate the best young international and college players. 

The lottery odds are based on where you finish the season. The team with the worst record of wins and loses gets the best chance to earn the first pick. Once that team is selected, they are no longer in the lottery for other picks. 

Only the first 4 draft picks are chosen in the lottery. After the first four selections are made, the remaining teams are allocated their draft picks based on what position they finished in the league (in worst to best order).

### Input
One file that contains three data sets.

The lottery odds are one of the inputs for the challenge this week but I'll let you format them how you want to with your Prep tool. The values in the table are for each team to land each spot. 

For example, for the 1st pick (column 1), each of the first 3 seeds have an even 14% chance on winning the first pick. The 14th team only has a 0.5% chance of winning the first pick. 

For the 2nd pick (column 2), the 13 teams who haven't won the 1st pick are then eligible to win the 2nd pick. The column has the relevant % chance of winning the pick. 

By the 5th pick, if the 1st seed hasn't successfully gained any of the previous picks, they would receive the 5th pick so for this challenge you don't need to worry about the odds below the 4th pick. 

### Requirements
The overall objective is to:
 - Create a workflow that will allocate the draft picks in a random manner based on the odds for each team 
 - The workflow should allocate each of the first 4 picks based on the lottery odds and then allocate all teams that didn't receive a slot to the remaining places in order

In [1]:
import pandas as pd
from random import choices

In [2]:
#Input the data
with pd.ExcelFile('WK27-Input.xlsx') as xlsx:
    seed = pd.read_excel(xlsx, 'Seeding')
    scaffold = pd.read_excel(xlsx, 'Scaffold')
    teams = pd.read_excel(xlsx, 'Teams')

In [3]:
seed.head()

,Seed,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0
0,1.0,14.0,13.4,12.7,12.0,47.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,14.0,13.4,12.7,12.0,27.8,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,14.0,13.4,12.7,12.0,14.8,26.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,12.5,12.2,11.9,11.5,7.2,25.7,16.7,2.2,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,10.5,10.5,10.6,10.5,2.2,19.6,26.7,8.7,0.6,NaN,NaN,NaN,NaN,NaN


In [4]:
scaffold.head()

,Scaffold
0,1.0
1,2.0
2,3.0
3,4.0
4,5.0


In [5]:
teams.head()

,Seed,Team
0,1.0,Houston
1,2.0,Detroit
2,3.0,Orlando
3,4.0,Oklahoma City
4,5.0,Cleveland


In [6]:
# melt pick probabilities into rows and convert to numbers
seed = pd.melt(seed, id_vars=['Seed'], var_name='pick', value_name='prob').dropna()
seed['prob'] = seed['prob'].astype(str).str.replace('>', '').astype(float)

In [7]:
seed.head()

,Seed,pick,prob
0,1.0,1.0,14.0
1,2.0,1.0,14.0
2,3.0,1.0,14.0
3,4.0,1.0,12.5
4,5.0,1.0,10.5


In [9]:
# select the first four picks by lottery
lottery_count = 4
lottery = []

for pick in range(lottery_count):
    
    # subset the probability table and randomly select a seed based on the weights
    prob_sub = seed.loc[(seed['pick']==pick+1) & ~seed['Seed'].isin(lottery)]
    lottery.append(choices(list(prob_sub['Seed']), weights=list(prob_sub['prob']))[0])

In [10]:
# add the remaining teams in seed order and join to the Teams list for final output
output = pd.DataFrame({'Actual Pick' : range(1,len(teams)+1),
                    'Seed' : lottery + [t for t in teams['Seed'].sort_values() if t not in lottery]})\
             .merge(teams, on='Seed')\
             .sort_values(by='Actual Pick')\
             .rename(columns={'Seed' : 'Original'})

In [11]:
output.head()

,Actual Pick,Original,Team
0,1,7.0,Toronto
1,2,3.0,Orlando
2,3,6.0,Minnesota
3,4,1.0,Houston
4,5,2.0,Detroit


In [12]:
output.to_csv('wk27-output.csv', index=False)